In [1]:
import scrapy
import requests
from scrapy import Selector

import yfinance as yf

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd

import time

In [2]:
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'

html = requests.get(url).text
sel = Selector(text = html)

nasdaq100_list = []

for n in range(2,104):
    ticker = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[2]'.format(n)).extract()[0].strip('</td>')
    nasdaq100_list.append(ticker)

In [4]:
%pprint

nasdaq100_list

Pretty printing has been turned OFF


['ATVI', 'ADBE', 'AMD', 'ALGN', 'GOOGL', 'GOOG', 'AMZN', 'AEP', 'AMGN', 'ADI', 'ANSS', 'AAPL', 'AMAT', 'ASML', 'TEAM', 'ADSK', 'ADP', 'BIDU', 'BIIB', 'BKNG', 'AVGO', 'CDNS', 'CDW', 'CERN', 'CHTR', 'CHKP', 'CTAS', 'CSCO', 'CTSH', 'CMCSA', 'CPRT', 'COST', 'CRWD', 'CSX', 'DXCM', 'DOCU', 'DLTR', 'EBAY', 'EA', 'EXC', 'FAST', 'FISV', 'FOXA', 'FOX', 'GILD', 'HON', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG', 'JD', 'KDP', 'KLAC', 'KHC', 'LRCX', 'LULU', 'MAR', 'MRVL', 'MTCH', 'MELI', 'FB', 'MCHP', 'MU', 'MSFT', 'MRNA', 'MDLZ', 'MNST', 'NTES', 'NFLX', 'NVDA', 'NXPI', 'ORLY', 'OKTA', 'PCAR', 'PAYX', 'PYPL', 'PTON', 'PEP', 'PDD', 'QCOM', 'REGN', 'ROST', 'SGEN', 'SIRI', 'SWKS', 'SPLK', 'SBUX', 'SNPS', 'TMUS', 'TSLA', 'TXN', 'TCOM', 'VRSN', 'VRSK', 'VRTX', 'WBA', 'WDAY', 'XEL', 'XLNX', 'ZM']

In [5]:
def print_company_year_perfomance(ticker_name):
    stock_ticker = yf.Ticker(ticker_name)
    history_df = stock_ticker.history(period = '1y').reset_index()

    history_df.loc[:,'year'] = [date.isocalendar()[0] for date in history_df['Date']]
    history_df.loc[:,'week'] = [date.isocalendar()[1] for date in history_df['Date']]

    weekly_history_df = history_df[['year','week','Close']].groupby(['year','week']).mean()
    weekly_history_df = weekly_history_df.reset_index()

    weekly_history_df.loc[:,'close'] = weekly_history_df['Close'].round(1)
    weekly_history_df = weekly_history_df.drop(columns = ['Close'])

    return print("Minimum close price: {},\nMaximum close price: {},\nYoY Growth: {},\nStandard Deviation: {},\nVariance: {}".\
        format(weekly_history_df['close'].min(), weekly_history_df['close'].max(),\
             round((weekly_history_df.loc[52]['close']-weekly_history_df.loc[0]['close'])/weekly_history_df.loc[0]['close'],3),
             round(weekly_history_df['close'].std(),2),
             round(weekly_history_df['close'].var(),2)))

In [17]:
print_company_year_perfomance('JD')

Minimum close price: 64.5,
Maximum close price: 105.5,
YoY Growth: -0.017,
Standard Deviation: 8.49,
Variance: 72.02


In [6]:
def performance_row(ticker_name):
    stock_ticker = yf.Ticker(ticker_name)
    history_df = stock_ticker.history(period = '1y', interval = '5m').reset_index()

    history_df.loc[:,'year'] = [date.isocalendar()[0] for date in history_df['Date']]
    history_df.loc[:,'week'] = [date.isocalendar()[1] for date in history_df['Date']]

    weekly_history_df = history_df[['year','week','Close']].groupby(['year','week']).mean()
    weekly_history_df = weekly_history_df.reset_index()

    weekly_history_df.loc[:,'close'] = weekly_history_df['Close'].round(1)
    weekly_history_df = weekly_history_df.drop(columns = ['Close'])

    performance_list = [ticker_name,\
    round((weekly_history_df.loc[52]['close']-weekly_history_df.loc[0]['close'])/weekly_history_df.loc[0]['close'],3),\
    round(weekly_history_df['close'].std(),2),\
    round(weekly_history_df['close'].var(),2)]

    return pd.Series(performance_list, index = ['ticker','yoy_growth','std','variance'])

In [7]:
my_portfolio_tickers = ['ABBV','BABA','JD','BSX']

In [ ]:
counter = 0
comparison_df = pd.DataFrame(columns = ['ticker','yoy_growth','std','variance'])

for ticker_name in my_portfolio_tickers:
    if ticker_name not in ['GOOG','AEP','ADI']:
        comparison_df = comparison_df.append(performance_row(ticker_name), ignore_index=True)
    else:
        pass
    print(ticker_name, end = ' ')
    time.sleep(3)

In [8]:
def yearly_history_by_week(ticker_name):
    history_df = yf.download(tickers=ticker_name, period='1y').reset_index()[['Date','Close']]

    history_df.loc[:,'year'] = [date.isocalendar()[0] for date in history_df['Date']]
    history_df.loc[:,'week'] = [date.isocalendar()[1] for date in history_df['Date']]

    weekly_history_df = history_df[['year','week','Close']].groupby(['year','week']).mean()
    weekly_history_df = weekly_history_df.reset_index()

    weekly_history_df.loc[:,'close'] = weekly_history_df['Close'].round(1)
    weekly_history_df = weekly_history_df.drop(columns = ['Close'])
    weekly_history_df = weekly_history_df.rename(columns = {'close': ticker_name.lower()})
    return weekly_history_df

In [9]:
history_df = yearly_history_by_week('EXC')

for company in my_portfolio_tickers:
    history_df = history_df.merge(yearly_history_by_week(company), left_on = ['year','week'], right_on = ['year','week'])
    print(company, end = ' ')

history_df.head(4)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
BSX 

,year,week,exc,abbv,baba,jd,bsx
0,2020,47,42.5,99.6,260.8,86.8,34.3
1,2020,48,42.1,103.8,276.1,88.8,33.6
2,2020,49,41.2,105.1,264.6,85.3,33.8
3,2020,50,41.2,107.2,264.7,82.3,34.0


In [11]:
history21_df = history_df[history_df['year'] >= 2021]

for company in history21_df.columns[2:]:
    history21_df.loc[:,company+'_%'] = history21_df[company].apply(lambda x: round(x/history21_df[company].to_list()[0]-1,3)*100)

history21_df.head(10)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,year,week,exc,abbv,baba,jd,bsx,exc_%,abbv_%,baba_%,jd_%,bsx_%
7,2021,1,41.8,106.3,231.8,89.9,36.3,0.0,0.0,0.0,0.0,0.0
8,2021,2,42.5,110.7,234.9,89.3,36.1,1.7,4.1,1.3,-0.7,-0.6
9,2021,3,42.9,111.7,258.9,94.1,36.9,2.6,5.1,11.7,4.7,1.7
10,2021,4,42.2,105.8,260.4,93.1,36.2,1.0,-0.5,12.3,3.6,-0.3
11,2021,5,43.0,106.0,263.0,94.7,37.6,2.9,-0.3,13.5,5.3,3.6
12,2021,6,43.1,104.9,266.7,97.7,39.0,3.1,-1.3,15.1,8.7,7.4
13,2021,7,42.1,105.4,267.4,105.5,37.7,0.7,-0.8,15.4,17.4,3.9
14,2021,8,39.8,107.4,247.0,96.1,39.2,-4.8,1.0,6.6,6.9,8.0
15,2021,9,39.5,107.1,235.4,93.9,38.8,-5.5,0.8,1.6,4.4,6.9
16,2021,10,41.7,107.4,234.4,87.6,39.2,-0.2,1.0,1.1,-2.6,8.0


In [16]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()

fig.add_trace(go.Scatter(x=history21_df['week'], y=history21_df['exc_%'], mode='lines', name='exc_%'))
fig.add_trace(go.Scatter(x=history21_df['week'], y=history21_df['abbv_%'], mode='lines', name='abbv_%'))
fig.add_trace(go.Scatter(x=history21_df['week'], y=history21_df['baba_%'], mode='lines', name='baba_%'))

fig.update_layout(height = 500, width = 1000)

fig.show()